In [12]:
import os
import cv2
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
import tensorflow as tf

# Define paths to your images and masks (change these to your actual paths)
image_dir = r'C:\Users\krraj\OneDrive\Desktop\5C\Data\TCGA_CS_4941_19960909\images'
mask_dir = r'C:\Users\krraj\OneDrive\Desktop\5C\Data\TCGA_CS_4941_19960909\masks'

# 1. Function to load all MRI images
def load_images(image_dir):
    images = []
    for img_file in glob.glob(os.path.join(image_dir, '*.tif')):  
        img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
        if img is not None:
            img = cv2.resize(img, (256, 256))  # Resize if needed (optional)
            images.append(img)
    return np.array(images)

# 2. Function to load all corresponding masks
def load_masks(mask_dir):
    masks = []
    for mask_file in glob.glob(os.path.join(mask_dir, '*.tif')):  
        mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)  # Load mask in grayscale
        if mask is not None:
            mask = cv2.resize(mask, (256, 256))  # Resize if needed (optional)
            masks.append(mask)
    return np.array(masks)

# Load images and masks
images = load_images(image_dir)
masks = load_masks(mask_dir)

print(f'Loaded {len(images)} images and {len(masks)} masks.')

# 3. Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) on images
def apply_clahe(images):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    processed_images = []
    for img in images:
        clahe_img = clahe.apply(img)  # Apply CLAHE to each image
        processed_images.append(clahe_img)
    return np.array(processed_images)

# Apply CLAHE to images
images_clahe = apply_clahe(images)

# 4. Normalize both images and masks
def normalize_images(images):
    return images / 255.0  # Normalize pixel values to range [0, 1]

images_norm = normalize_images(images_clahe)
masks_norm = normalize_images(masks)

# 5. Reshape the data to match the model input shape
X_train, X_test, y_train, y_test = train_test_split(images_norm, masks_norm, test_size=0.2, random_state=42)
X_train = X_train[..., np.newaxis]  # Shape: (batch_size, 256, 256, 1)
X_test = X_test[..., np.newaxis]    # Shape: (batch_size, 256, 256, 1)
y_train = y_train[..., np.newaxis]  # Shape: (batch_size, 256, 256, 1)
y_test = y_test[..., np.newaxis]    # Shape: (batch_size, 256, 256, 1)

print(f'Training set: {X_train.shape}, Test set: {X_test.shape}')

def dice_coefficient(y_true, y_pred):
    smooth = 1e-6
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# 7. Define a simple U-Net model (can be replaced with Nested U-Net or Attention U-Net)
def build_unet_model(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    up4 = UpSampling2D(size=(2, 2))(conv3)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(merge4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    model = Model(inputs, outputs)
    return model

# Build models (Nested U-Net and Attention U-Net can be defined similarly)
nested_unet_model = build_unet_model((256, 256, 1))
attention_unet_model = build_unet_model((256, 256, 1))  # Placeholder for Attention U-Net

# 8. Compile the models
nested_unet_model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryCrossentropy(), metrics=[dice_coefficient])
attention_unet_model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryCrossentropy(), metrics=[dice_coefficient])


# 9. Train the models
history_nested_unet = nested_unet_model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_test, y_test))
history_attention_unet = attention_unet_model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_test, y_test))

# 10. Evaluate the models
nested_unet_eval = nested_unet_model.evaluate(X_test, y_test)
attention_unet_eval = attention_unet_model.evaluate(X_test, y_test)

print(f"Nested U-Net Evaluation: {nested_unet_eval}")
print(f"Attention U-Net Evaluation: {attention_unet_eval}")


Loaded 23 images and 23 masks.
Training set: (18, 256, 256, 1), Test set: (5, 256, 256, 1)
Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - dice_coefficient: 0.0203 - loss: 0.6836 - val_dice_coefficient: 2.7127e-04 - val_loss: 0.6614
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - dice_coefficient: 0.0288 - loss: 0.6383 - val_dice_coefficient: 2.3108e-04 - val_loss: 0.5590
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - dice_coefficient: 0.0178 - loss: 0.4827 - val_dice_coefficient: 6.1407e-05 - val_loss: 0.2812
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - dice_coefficient: 0.0027 - loss: 0.2251 - val_dice_coefficient: 5.7712e-08 - val_loss: 0.0675
Epoch 5/20
